In [ ]:
#Reload all modules (except those excluded by %aimport) every time before executing the Python code typed.
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from __future__ import absolute_import
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
import pickle
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
import string #print (string.punctuation)
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import preprocessing

#Classifiers
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

#helpers
from pprint import pprint
from time import time
import logging,re

from sklearn.metrics import classification_report,confusion_matrix
import pandas as pd
import nltk
import csv
from nltk.corpus import stopwords
from string import punctuation
#initializersfrom keras import initializers
from keras.callbacks import CSVLogger
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM,Bidirectional,TimeDistributed,InputLayer,GRU
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, Conv2D, MaxPool2D, Reshape, Flatten
from keras.layers import SpatialDropout1D, Dense
from keras.optimizers import Adam
from keras.models import load_model
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Merge
from keras.layers.convolutional import Convolution2D,MaxPooling2D,Convolution1D,MaxPooling1D,UpSampling2D, ZeroPadding2D

In [ ]:
#import sys
#!{sys.executable} -m pip install tensorflow
#!{sys.executable} -m pip install keras
#!{sys.executable} -m pip install pydot
#!{sys.executable} -m pip install graphviz
#!{sys.executable} -m pip install mlxtend  

In [ ]:
from workflow.data import get_trademark_data
from workflow.data import prepare_Xy

from workflow.utilities import save_object
from workflow.utilities import load_object

from workflow.fasttextwrap import fasttextClassifier

from classifiers.kerasTextClassifier import create_bi_lstm_model
from classifiers.kerasTextClassifier import create_model
from workflow.train import getStopWords
from workflow.train import trainFastTextModel
from workflow.train import cleanData
from workflow.train import clean_text

from workflow.test import test_pred

In [ ]:
'''
##########################################
TODO
-add ability for stratified sampling
##########################################
'''
random_state = 47
test_size = .99
stopwords_file = 'customstopwords.csv'

In [ ]:
stopwordsList = getStopWords()
en_stop = set(stopwords.words('english'))
en_stop.update(['.','(','(', ')','u',',', '"', "'", '?', '!', ':', ';' ,'(', ')', '[', ']', '{', '}', '&', '/', '-', '+', '*',\
                       "..", '...', '....', "+...", "-...", ",...", "'...", "!!!", "&...", "(...", ")...", "]...", "/...", "(+)...", "),...",\
                       "),", ").", "):", ")-", "))", "])", ".)", "!)", "')", "][", '").', '")/', '",', '"-', '")', '"/', './', '--',\
                       "#:", "(+)", "($", "-$", "/","+/",",", "+-", "(#", "''"])
stopWords = en_stop.union(stopwordsList)
#print (stopWords)

In [ ]:
'''
##########################################
TODO
-Add ability to select subset of labels
-Add ability to set data_dir (various impacts throughout files)
##########################################
'''
data = get_trademark_data(
    statement_url='https://bulkdata.uspto.gov/data/trademark/casefile/economics/2016/statement.csv.zip',
    design_url='https://bulkdata.uspto.gov/data/trademark/casefile/economics/2016/design_search.csv.zip',
    sample_size=1, #1 means 100% of the data. Lower it (.1) to run faster when testing script
    test_size=0.99, #Percentage of data to set aside for testing
    filename='trademark_data_updated_Naresh.csv', #save retrieved data as this name, if exists will load .csv instead of url download
    data_dir='', #save retrieved data in this folder
    force_download=False #redownload from url and overwrite and saved .csv
    )

data.head()

X,y = prepare_Xy(data)

In [ ]:
'''
##########################################
TODO
-add analysis jupyter workbook

##########################################
'''
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y)
print (len (mlb.classes_))
#print (mlb.classes_)

In [ ]:
#for value in mlb.classes_:
    #print(value)

In [ ]:
import copy
copyofX = copy.deepcopy(X)
#copyofX
len(copyofX)

In [ ]:
def __remove_stop_words(text):
    return ' '.join([word for word in text.split() if word.lower() not in stopWords])

shortword = re.compile(r'\W*\b\w{1,3}\b')
myList= []
for sentence in copyofX :
    #print (sentence)
    sentence= shortword.sub('', sentence)
    sentence = __remove_stop_words(sentence)
    sentence=re.sub('([^\s\w]|_)+',' ',sentence).strip() 
    #sentence=re.sub('[?!\\,]+','',sentence).strip() #remove comma from sentence
    #sentence=re.sub('[?!\\.]+','',sentence).strip() #remove punctuation from sentence because we don't need it anymore
    #sentence=re.sub('[?!\\"]+','',sentence).strip() #remove quotes from sentence
    #sentence=re.sub('[?!\\-]+','',sentence).strip() #remove dash from sentence
    sentence=re.sub('\s+',' ',sentence)	# remove multiple spaces
    myList.append(sentence.lower())
    #print (sentence.lower())
print (len(myList))
print (myList[:1])

In [ ]:
'''
##########################################
TODO
-add ability to have multiple model types
-seperate out preprocessing and vect from fast text
-add other models
-select variables

##########################################
'''

# import importlib
# importlib.reload(workflow.fasttextwrap)
# from workflow.fasttextwrap import fasttextClassifier
X_train, X_test, y_train, y_test = train_test_split(myList, y, test_size=test_size, random_state=random_state)
print(X_train[0:2])
print(X_test[0:2])
#print (y_train[0:2])
#print (y_test[0:2])


In [ ]:
MAX_SEQUENCE_LENGTH = 200
MAX_NB_WORDS = 50000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 1

# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True)
tokenizer.fit_on_texts(myList)
sequences = tokenizer.texts_to_sequences(myList)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
#labels = to_categorical(np.asarray(labels)) #aaron removed
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', y.shape)

In [ ]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
y = y[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = y[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = y[-num_validation_samples:]

print('shape of:', x_train.shape)
print('shape of:',y_train.shape)
print('shape of:',x_val.shape)
print('shape of:',y_val.shape)

In [ ]:
import os
from keras.layers import Embedding

BASE_DIR = ''
#GLOVE_DIR = os.path.join(BASE_DIR, 'glove_embedding')
#TEXT_DATA_DIR = os.path.join(BASE_DIR, 'glove.6B.300d.txt')
GLOVE_DIR = os.path.join(BASE_DIR, 'fasttext_word_representations')
TEXT_DATA_DIR = os.path.join(BASE_DIR, 'latest_tm_markdesc_300.vec')
MAX_SEQUENCE_LENGTH = 200
MAX_NB_WORDS = 50000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2

# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'latest_tm_markdesc_300.vec'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


embedding_layer = Embedding(len(word_index) + 1,
                           EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
len(embedding_matrix)

In [ ]:
from workflow.data import get_trademark_data
from workflow.data import prepare_Xy

pred_data = get_trademark_data(
    statement_url='https://bulkdata.uspto.gov/data/trademark/casefile/economics/2016/statement.csv.zip',
    design_url='https://bulkdata.uspto.gov/data/trademark/casefile/economics/2016/design_search.csv.zip',
    sample_size=1, #1 means 100% of the data. Lower it (.1) to run faster when testing script
    test_size=.2, #Percentage of data to set aside for testing
    filename='sashi_1381_test_mark_desc_output.csv', #save retrieved data as this name, if exists will load .csv instead of url download
    data_dir='', #save retrieved data in this folder
    force_download=False #redownload from url and overwrite and saved .csv
    )

pred_data.head()

pred_X,pred_y = prepare_Xy(pred_data)
print (len (pred_y))

In [ ]:
mlb = MultiLabelBinarizer()
pred_y = mlb.fit_transform(pred_y)
#print (mlb.classes_)

In [ ]:
import copy
pred_copyofX = copy.deepcopy(pred_X)
#copyofX
len(pred_copyofX)

In [ ]:
def __remove_stop_words(text):
    return ' '.join([word for word in text.split() if word.lower() not in stopWords])

shortword = re.compile(r'\W*\b\w{1,3}\b')
predList= []
for sentence in pred_copyofX :
    #print (sentence)
    if not (pd.isnull(sentence)):
        sentence= shortword.sub('', sentence)
        sentence = __remove_stop_words(sentence)
        sentence=re.sub('([^\s\w]|_)+',' ',sentence).strip() 
        #sentence=re.sub('[?!\\,]+','',sentence).strip() #remove comma from sentence
        #sentence=re.sub('[?!\\.]+','',sentence).strip() #remove punctuation from sentence because we don't need it anymore
        #sentence=re.sub('[?!\\"]+','',sentence).strip() #remove quotes from sentence
        #sentence=re.sub('[?!\\-]+','',sentence).strip() #remove dash from sentence
        sentence=re.sub('\s+',' ',sentence)	# remove multiple spaces
        predList.append(sentence.lower())
        #print (sentence.lower())
    else:
        #print (sentence)
        sentence = " "
        predList.append(sentence)
print (len(predList))

In [ ]:
def bigru_cnn():
    print('Loading model.')
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    units = 128
    conv_filters = 32
    x = Dropout(0.2)(embedded_sequences)
    x = Bidirectional(GRU(
        units,
        dropout=0.2,
        recurrent_dropout=0.2,
        return_sequences=True))(x)
    x = Reshape((2 * MAX_SEQUENCE_LENGTH, units, 1))(x)
    x = Conv2D(conv_filters, (3, 3))(x)

    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    preds = Dense(y_train.shape[1], activation='sigmoid')(x) # for multilabel classification # length of labels
    model = Model(sequence_input, preds)
    return model

In [ ]:
bigru_cnn_model = bigru_cnn()
bigru_cnn_model.load_weights('200seq_300d_1381_BI_GRU_tm_embedding.hdf5')

In [ ]:
def __remove_stop_words(text):
    return ' '.join([word for word in text.split() if word.lower() not in stopWords])

In [ ]:
from ast import literal_eval

dtype1 = {'serial_no':np.dtype('U'), 'statement_text':np.dtype('U')}
data=pd.read_csv("sashi_1381_test_mark_desc_output.csv",dtype=dtype1,usecols=(1,2,3), converters={'design_search_cd':literal_eval})
#print (data)

#statement_data = pd.read_csv("mark_desc_output.csv")
#print (statement_data)
#subcolumns = statement_data[['serial_no','statement_text','design_search_cd']]
#dsc = list(subcolumns['design_search_cd'])
# add the leading 0 to search codes that had it removed
#dsc = list (subcolumns['design_search_cd'].astype(str).str.zfill(6))

#print (dsc[0:3])
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(list(data['design_search_cd']))
#print (y)
print (len (mlb.classes_))

In [ ]:
#for value in mlb.classes_:
    #print(value)

In [ ]:
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical
    
test_txt = ['stylized lion mermaid']
#test_txt = ['stylized lion circle', 'mermaid cup vapor']

MAX_SEQUENCE_LENGTH = 200
MAX_NB_WORDS = 50000
VALIDATION_SPLIT = 1
print (len(myList))
print (len(predList))
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True)
tokenizer.fit_on_texts(myList)
sequences = tokenizer.texts_to_sequences(predList)
#print (sequences)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
#print (data)
#labels = to_categorical(np.asarray(labels)) #aaron removed
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', y.shape)

indices = np.arange(data.shape[0])
#np.random.shuffle(indices)
data = data[indices]
y1 = pred_y[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = y1[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = y1[-num_validation_samples:]

print('shape of x_train:', x_train.shape)
print('shape of y_train:',y_train.shape)
print('shape of x_val:',x_val.shape)
print('shape of y_val:',y_val.shape)


test_pred = bigru_cnn_model.predict(x_val, verbose=0)


In [ ]:
print (predList[:1])
print (test_pred.shape)
#print (new_test_pred.shape)
print (len(test_pred))

In [ ]:
import numpy as np

print (len(test_pred))
save_object(mlb, 'sashi_test_data/sashi_data_mlb.sav')
save_object(test_pred, 'sashi_test_data/sashi_data_predictions.sav')
with open("sashi_test_data/sashi_data_predicted1381designcodes.txt", "w") as new_text_file:
    for x1 in range(len(test_pred)):
        #print (mlb.classes_[np.nonzero(test_pred[x1])[0]])
        x1 = mlb.classes_[np.nonzero(test_pred[x1])[0]]
        new_text_file.write(str(x1))

In [ ]:
prediction_prob=.3
with open("sashi_test_data/sashi_data_1381_predcodes_greater_0.3.txt", "w") as new_text_file:
    new_test_pred = np.where (test_pred > prediction_prob, 1, 0)
    for x2 in range(len(new_test_pred)):
        x2 = mlb.classes_[np.nonzero(new_test_pred[x2])[0]]
        new_text_file.write(str(x2))
        new_text_file.write(",")

In [ ]:
print(classification_report(y_val, new_test_pred, target_names=mlb.classes_)) #predict_proba scores by class

In [ ]:
print(classification_report(y_val, new_test_pred, target_names=mlb.classes_)) #predict_proba scores by class

In [ ]:
print(classification_report(y_val, new_test_pred, target_names=mlb.classes_)) #predict_proba scores by class

In [ ]:
def __remove_stop_words(text):
    return ' '.join([word for word in text.split() if word.lower() not in stopWords])

In [ ]:
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical
    
test_txt = ['one star with four points ']
#test_txt = ['THE TRADE-MARK IS OF AN OVAL SHAPE, HAVING A MARROON BACKGROUND WITH THE LETTERS BEING SOLID GOLD IN COLOR']
#test_txt = ['THE MARK COMPRISES A FANCIFUL ARRANGEMENT OF LETTER COMPRISING THE WORD CHYPRE OVER A DESIGN']
test_txt = __remove_stop_words(test_txt[0])
test_txt=re.sub('([^\s\w]|_)+',' ',test_txt).strip() 
test_txt=re.sub('\s+',' ',test_txt)	# remove multiple spaces
test_txt = [str (test_txt.lower())]
print (test_txt)
#test_txt = ['stylized lion circle', 'mermaid cup vapor']
prediction_prob=.3

MAX_SEQUENCE_LENGTH = 200
MAX_NB_WORDS = 50000
VALIDATION_SPLIT = 1
print (len(myList))
print (len(test_txt))
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True)
tokenizer.fit_on_texts(myList)
prediction_sequences = tokenizer.texts_to_sequences(test_txt)
#print (sequences)
prediction_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(prediction_word_index))

prediction_data = pad_sequences(prediction_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print (prediction_data)
#labels = to_categorical(np.asarray(labels)) #aaron removed
print('Shape of data tensor:', prediction_data.shape)
print('Shape of label tensor:', y.shape)

indices = np.arange(prediction_data.shape[0])
#np.random.shuffle(indices)
prediction_data = prediction_data[indices]
prediction_y1 = pred_y[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

prediction_x_train = prediction_data[:-num_validation_samples]
prediction_y_train = prediction_y1[:-num_validation_samples]
prediction_x_val = prediction_data[-num_validation_samples:]
prediction_y_val = prediction_y1[-num_validation_samples:]

print('shape of x_train:', prediction_x_train.shape)
print('shape of y_train:',prediction_y_train.shape)
print('shape of x_val:',prediction_x_val.shape)
print('shape of y_val:',prediction_y_val.shape)


test_prediction = bigru_cnn_model.predict(prediction_x_val, verbose=0)
new_test_prediction = np.where (test_prediction > prediction_prob, 1, 0)
print (new_test_prediction)
new_predictions_list = [x.replace(" ", ",") for x in mlb.classes_[np.nonzero(new_test_prediction)[1]]]
print (new_predictions_list)